<a href="https://colab.research.google.com/github/GermannM3/model_sf/blob/master/Kenga_Dual_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone -b master https://github.com/GermannM3/model_sf.git

Cloning into 'model_sf'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 222 (delta 35), reused 6 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (222/222), 660.56 KiB | 4.97 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [4]:
%cd model_sf

/content/model_sf


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.5/669.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
import matplotlib.pyplot as plt

# Импортируем модули для обучения и инференса
from src.neural.models.kenga import KengaConfig, KengaModel
from src.neural.training import KengaTrainer
from src.neural.data import TextDataset
from src.utils.tokenizer import Tokenizer

from src.neural.models.kenga_s import KengaSConfig, KengaSModel
from src.neural.training_kenga_s import KengaSTrainer

def generate_tokens(files, tokenizer):
    tokens = []
    for file in files:
        with open(file, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:
                    t = tokenizer.encode(line)
                    if t:
                        tokens.append(t)
    return tokens

# Создаем директории и файлы с данными
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)
train_files = ["data/train.txt"]
val_files = ["data/val.txt"]
for file in train_files + val_files:
    if not os.path.exists(file):
        with open(file, "w", encoding="utf-8") as f:
            f.write("This is a sample sentence.\nThis is another sentence.\n")

# Инициализируем токенизатор, обучаем его и сохраняем
tokenizer = Tokenizer()
tokenizer.train(train_files)
tokenizer.save("models", "kenga")

train_tokens = generate_tokens(train_files, tokenizer)
val_tokens = generate_tokens(val_files, tokenizer)

train_dataset = TextDataset(train_tokens)
val_dataset = TextDataset(val_tokens)

# Инициализация модели Kenga (MML) и тренера
config = KengaConfig()
config.vocab_size = tokenizer.vocab_size
model_mml = KengaModel(config)
trainer_mml = KengaTrainer(model_mml, train_dataset, val_dataset, {
    "lr": 1e-4,
    "batch_size": 4,
    "epochs": 5,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "model_path": "models/kenga.pth"
})

# Инициализация модели KengaS (SML) и тренера
config_s = KengaSConfig()
config_s.vocab_size = tokenizer.vocab_size
model_sml = KengaSModel(config_s)
trainer_sml = KengaSTrainer(model_sml, train_dataset, val_dataset, {
    "lr": 1e-4,
    "batch_size": 4,
    "epochs": 5,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "model_path": "models/kengaS.pth"
})

# Запускаем параллельное обучение обеих моделей
import threading
t1 = threading.Thread(target=trainer_mml.train)
t2 = threading.Thread(target=trainer_sml.train)
t1.start()
t2.start()
t1.join()
t2.join()

# После обучения выводим тестовый инференс
model_mml.eval()
model_sml.eval()
input_ids = torch.randint(0, tokenizer.vocab_size, (1, 10))
mml_output = model_mml(input_ids)
sml_output = model_sml(input_ids)
print("Kenga (MML) Output:", mml_output)
print("KengaS (SML) Output:", sml_output)

# Выводим графики прогресса обучения (если тренеры записали историю потерь)
if trainer_mml.train_loss_history and trainer_sml.train_loss_history:
    epochs_mml = range(1, len(trainer_mml.train_loss_history) + 1)
    epochs_sml = range(1, len(trainer_sml.train_loss_history) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1,2,1)
    plt.plot(epochs_mml, trainer_mml.train_loss_history, label='Train Loss (MML)')
    plt.plot(epochs_mml, trainer_mml.val_loss_history, label='Val Loss (MML)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Kenga (MML) Training Progress')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs_sml, trainer_sml.train_loss_history, label='Train Loss (SML)')
    plt.plot(epochs_sml, trainer_sml.val_loss_history, label='Val Loss (SML)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('KengaS (SML) Training Progress')
    plt.legend()

    plt.show()


In [ ]:
# Если настроены креденциалы Git, можно отправить чекпоинты в GitLab
!git config --global user.email "your_email@example.com"
!git config --global user.name "Your Name"
!git add models/
!git commit -m "Обновленные чекпоинты и графики обучения после тренировки в Colab"
!git push gitlab main